### XGBoost パラメータチューニング

In [1]:
import numpy as np
import pandas as pd

import xgboost
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error, make_scorer, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV


In [2]:
# pickle ファイルから読み込み
train_pkl = pd.read_pickle('./train.pk2')

In [3]:
train_pkl.shape

(850, 9)

In [4]:
train_pkl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 850 entries, 0 to 849
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   T_Bil     850 non-null    float16
 1   D_Bil     850 non-null    float16
 2   ALP       850 non-null    float32
 3   ALT_GPT   850 non-null    float32
 4   AST_GOT   850 non-null    float32
 5   TP        850 non-null    float16
 6   Alb       850 non-null    float16
 7   AG_ratio  850 non-null    float16
 8   disease   850 non-null    int64  
dtypes: float16(5), float32(3), int64(1)
memory usage: 25.0 KB


In [67]:
# 訓練とテストデータに分割
train, test = train_test_split(train_pkl, test_size=0.2, random_state=42)

In [68]:
# ターゲットと特徴量の分割
train_X = train.iloc[:, :-1].values
train_y = train.disease.values

### 検証するパラメータの設定

In [69]:
grid_param1 = {
    'max_depth': [i for i in range(3, 11, 2)],
#     'min_child_weight': [1, 2, 3, 4, 6, 10],
#     'gamma': [0.0, 0.1, 0.2, 2.0, 10.0],
#     'subsample': [0.5, 0.6, 0.7, 0.8, 1.0],
#     'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 1.0],
#     'reg_alpha': [1, 0.1, 0.01, 0.001],
#     'reg_lambda': [1, 0.1, 0.01, 0.001],
    'random_state': [42]
}


### グリッドサーチ

In [70]:
gs = GridSearchCV(estimator=XGBClassifier({'verbosity':0}), param_grid=grid_param1, scoring='accuracy', cv=5, return_train_score=False)

In [71]:
gs.fit(train_X, train_y)

GridSearchCV(cv=5, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth={'verbosity': 0},
                                     min_child_weight=1, missing=None,
                                     n_estimators=100, n_jobs=1, nthread=None,
                                     objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=None,
             param_grid={'max_depth': [3, 5, 7, 9], 'random_state': [42]},
             pre_dispatch='2*n_jobs', refit=True, 

In [72]:
gs.best_params_

{'max_depth': 5, 'random_state': 42}

In [73]:
grid_param2 = {
    'max_depth': [5],
    'min_child_weight': [i for i in range(1, 11)],
#     'gamma': [0.0, 0.1, 0.2, 2.0, 10.0],
#     'subsample': [0.5, 0.6, 0.7, 0.8, 1.0],
#     'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 1.0],
#     'reg_alpha': [1, 0.1, 0.01, 0.001],
#     'reg_lambda': [1, 0.1, 0.01, 0.001],
    'random_state': [42]
}


In [74]:
gs = GridSearchCV(estimator=XGBClassifier({'verbosity':0}), param_grid=grid_param2, scoring='accuracy', cv=5, return_train_score=False)

In [75]:
gs.fit(train_X, train_y)

GridSearchCV(cv=5, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth={'verbosity': 0},
                                     min_child_weight=1, missing=None,
                                     n_estimators=100, n_jobs=1, nthread=None,
                                     objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=None,
             param_grid={'max_depth': [5],
                         'min_child_weight': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    

In [76]:
gs.best_score_

0.8632352941176471

In [77]:
gs.best_params_

{'max_depth': 5, 'min_child_weight': 1, 'random_state': 42}

In [78]:
gs.cv_results_['mean_test_score']

array([0.86323529, 0.85882353, 0.85882353, 0.85882353, 0.85      ,
       0.85441176, 0.84852941, 0.85441176, 0.85147059, 0.84852941])

In [79]:
grid_param3 = {
    'max_depth': [5],
    'min_child_weight': [1],
    'gamma': [0.0, 0.1, 0.2, 2.0, 5.0, 10.0],
#     'subsample': [0.5, 0.6, 0.7, 0.8, 1.0],
#     'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 1.0],
#     'reg_alpha': [1, 0.1, 0.01, 0.001],
#     'reg_lambda': [1, 0.1, 0.01, 0.001],
    'random_state': [42]
}


In [80]:
gs = GridSearchCV(estimator=XGBClassifier({'verbosity':0}), param_grid=grid_param3, scoring='accuracy', cv=5, return_train_score=False)

In [81]:
gs.fit(train_X, train_y)

GridSearchCV(cv=5, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth={'verbosity': 0},
                                     min_child_weight=1, missing=None,
                                     n_estimators=100, n_jobs=1, nthread=None,
                                     objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=None,
             param_grid={'gamma': [0.0, 0.1, 0.2, 2.0, 5.0, 10.0],
                         'max_depth': [5], 'min_child_weig

In [82]:
gs.best_score_

0.8632352941176471

In [83]:
gs.best_params_

{'gamma': 0.0, 'max_depth': 5, 'min_child_weight': 1, 'random_state': 42}

In [84]:
gs.cv_results_['mean_test_score']

array([0.86323529, 0.85735294, 0.85735294, 0.84852941, 0.83382353,
       0.83823529])

In [85]:
grid_param4 = {
    'max_depth': [5],
    'min_child_weight': [1],
    'gamma': [0.0],
    'subsample': [0.5, 0.6, 0.7, 0.8, 1.0],
#     'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 1.0],
#     'reg_alpha': [1, 0.1, 0.01, 0.001],
#     'reg_lambda': [1, 0.1, 0.01, 0.001],
    'random_state': [42]
}


In [86]:
gs = GridSearchCV(estimator=XGBClassifier({'verbosity':0}), param_grid=grid_param4, scoring='accuracy', cv=5, return_train_score=False)

In [87]:
gs.fit(train_X, train_y)

GridSearchCV(cv=5, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth={'verbosity': 0},
                                     min_child_weight=1, missing=None,
                                     n_estimators=100, n_jobs=1, nthread=None,
                                     objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=None,
             param_grid={'gamma': [0.0], 'max_depth': [5],
                         'min_child_weight': [1], 'random_state': 

In [88]:
gs.best_score_

0.8632352941176471

In [89]:
gs.best_params_

{'gamma': 0.0,
 'max_depth': 5,
 'min_child_weight': 1,
 'random_state': 42,
 'subsample': 1.0}

In [90]:
gs.cv_results_['mean_test_score']

array([0.84117647, 0.85735294, 0.85882353, 0.85588235, 0.86323529])

In [91]:
grid_param5 = {
    'max_depth': [5],
    'min_child_weight': [1],
    'gamma': [0.0],
    'subsample': [1.0],
    'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 1.0],
#     'reg_alpha': [1, 0.1, 0.01, 0.001],
#     'reg_lambda': [1, 0.1, 0.01, 0.001],
    'random_state': [42]
}


In [92]:
gs = GridSearchCV(estimator=XGBClassifier({'verbosity':0}), param_grid=grid_param5, scoring='accuracy', cv=5, return_train_score=False)

In [93]:
gs.fit(train_X, train_y)

GridSearchCV(cv=5, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth={'verbosity': 0},
                                     min_child_weight=1, missing=None,
                                     n_estimators=100, n_jobs=1, nthread=None,
                                     objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=None,
             param_grid={'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 1.0],
                         'gamma': [0.0], 'max_depth':

In [94]:
gs.best_score_

0.8632352941176471

In [95]:
gs.best_params_

{'colsample_bytree': 1.0,
 'gamma': 0.0,
 'max_depth': 5,
 'min_child_weight': 1,
 'random_state': 42,
 'subsample': 1.0}

In [96]:
gs.cv_results_['mean_test_score']

array([0.85294118, 0.85294118, 0.85735294, 0.85588235, 0.86323529])

In [97]:
grid_param6 = {
    'max_depth': [5],
    'min_child_weight': [1],
    'gamma': [0.0],
    'subsample': [1.0],
    'colsample_bytree': [1.0],
    'reg_alpha': [1, 0.1, 0.01, 0.001, 0.0001],
#     'reg_lambda': [1, 0.1, 0.01, 0.001],
    'random_state': [42]
}


In [98]:
gs = GridSearchCV(estimator=XGBClassifier({'verbosity':0}), param_grid=grid_param6, scoring='accuracy', cv=5, return_train_score=False)

In [99]:
gs.fit(train_X, train_y)

GridSearchCV(cv=5, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth={'verbosity': 0},
                                     min_child_weight=1, missing=None,
                                     n_estimators=100, n_jobs=1, nthread=None,
                                     objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lam...
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=None,
             param_grid={'colsample_bytree': [1.0], 'gamma': [0.0],
                         'max_depth': [5], 'min_child_weight

In [100]:
gs.best_score_

0.8632352941176471

In [101]:
gs.best_params_

{'colsample_bytree': 1.0,
 'gamma': 0.0,
 'max_depth': 5,
 'min_child_weight': 1,
 'random_state': 42,
 'reg_alpha': 0.0001,
 'subsample': 1.0}

In [102]:
gs.cv_results_['mean_test_score']

array([0.86176471, 0.85882353, 0.85441176, 0.86176471, 0.86323529])

In [39]:
grid_param7 = {
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 2, 3, 5],
    'gamma': [0.0, 0.1, 1.0],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 1.0],
    'reg_alpha': [1, 0.1, 0.01, 0.0001],
    'reg_lambda': [0.1, 1],
    'random_state': [42]
}


In [40]:
gs = GridSearchCV(estimator=XGBClassifier({'verbosity':0}), param_grid=grid_param7, scoring='accuracy', cv=5, return_train_score=False)

In [41]:
gs.fit(train_X, train_y)

GridSearchCV(cv=5, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth={'verbosity': 0},
                                     min_child_weight=1, missing=None,
                                     n_estimators=100, n_jobs=1, nthread=None,
                                     objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lam...
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=None,
             param_grid={'colsample_bytree': [0.6, 0.7, 0.8, 1.0],
                         'gamma': [0.0, 0.1, 1.0], 'max_depth': [3, 5, 7],
                         'min_child_weight': [1, 2, 3, 5], 'random_

In [42]:
gs.best_score_

0.8705882352941178

In [43]:
gs.best_params_

{'colsample_bytree': 0.6,
 'gamma': 0.1,
 'max_depth': 5,
 'min_child_weight': 3,
 'random_state': 42,
 'reg_alpha': 0.1,
 'reg_lambda': 1,
 'subsample': 1.0}

In [44]:
# gs.cv_results_['mean_test_score']

In [45]:
XGB = XGBClassifier(**gs.best_params_)

In [46]:
XGB = XGB.fit(train_X, train_y)

In [47]:
XGB.feature_importances_

array([0.27204463, 0.08743344, 0.08017332, 0.18823224, 0.16463326,
       0.05658108, 0.05532863, 0.09557338], dtype=float32)

In [48]:
sorted(
    zip(map(lambda x: round(x, 3), XGB.feature_importances_), train.iloc[:, :-1].columns),
    reverse=True)

[(0.272, 'T_Bil'),
 (0.188, 'ALT_GPT'),
 (0.165, 'AST_GOT'),
 (0.096, 'AG_ratio'),
 (0.087, 'D_Bil'),
 (0.08, 'ALP'),
 (0.057, 'TP'),
 (0.055, 'Alb')]

### テストデータで予測

In [106]:
# ターゲットと特徴量の分割
test_x = test.iloc[:, :-1]
test_y = test.disease.values

In [107]:
test_x.shape, test_y.shape

((170, 8), (170,))

In [108]:
pred_y = XGB.predict(test_x)

In [109]:
confusion_matrix(test_y, pred_y)

array([[88,  5],
       [14, 63]], dtype=int64)

In [110]:
accuracy_score(test_y, pred_y)

0.888235294117647

### 検証データで予測

In [54]:
# 検証データ読み込み
valid = pd.read_pickle('./test.pk2')

In [55]:
valid.shape

(350, 8)

In [56]:
# ID の保存
# valid_pass = valid.id.values

In [57]:
# valid_X = valid.iloc[:, 1:].values
valid_X = valid.copy()

In [58]:
valid_X.shape, train_X.shape

((350, 8), (680, 8))

In [59]:
pred_valid_y = XGB.predict(valid_X)

In [60]:
pred_valid_y.shape

(350,)

In [61]:
# type(valid_pass), type(pred_valid_y)

In [62]:
result_df = pd.DataFrame(pred_valid_y)

In [66]:
result_df.to_csv("./XGB_P_0001.csv",  header=False)

In [65]:
result_df.head(10)

,0
0,0
1,0
2,0
3,0
4,0
5,0
6,1
7,1
8,1
9,0
